## Input of data

In [4]:
from FUNCTION_single_doc_prep_to_embed import single_doc_prep_to_embed
import pandas as pd

In [12]:
LST_pdf_docs = ["Certifications Key to Unlocking DoD8140 Compliance.pdf",
                "Deepfake Engineering_ A New Concern for the C-Suite.pdf",
                "Diving into DeepSeek.pdf",
                "EU Cyber Solidarity Act - What You Need to Know.pdf",
                "ISC2 Spotlight_ GRC and the NIST 2.0 Cyber Security Framework.pdf",
                "ISC2 Welcomes the EU Action Plan for the Cybersecurity of Hospitals and Healthcare Providers.pdf",
                "Navigating Identity Governance and Administration_ Experiences in the Real World.pdf",
                "The Persistent Threat of Social Engineering.pdf"]
LST_num_sentence_chunk_size = [7,7,7,5,7,7,7,7]
LST_doc_alias_name = ["DoD8140 Compliance",
                      "Deepfake Eng",
                      "DeepSeek",
                      "EU Cyber Act",
                      "GRC & NIST 2.0 Framework",
                      "EU Plan Cybersec of Hospitals",
                      "Identity Governance & Admin",
                      "Social Eng Persistent Threat"]

In [ ]:
df = single_doc_prep_to_embed(PDF_PATH="Am I Logging the Right AWS Log Sources_.pdf",
                               NUM_SENTENCE_CHUNK_SIZE=7,
                               DOCUMENT_NAME="log the right AWS Log Sources")

In [14]:
for i in range(len(LST_pdf_docs)):
    df = pd.concat([df, single_doc_prep_to_embed(PDF_PATH=LST_pdf_docs[i],
                                           NUM_SENTENCE_CHUNK_SIZE=LST_num_sentence_chunk_size[i],
                                           DOCUMENT_NAME=LST_doc_alias_name[i])
    ]
    )

First pages's info (including the text): 

{'page_number': 1, 'page_char_count': 1562, 'page_word_count': 243, 'page_sentence_count_raw': 9, 'page_token_count': 390.5, 'text': 'Earlier this year, ISC2 announced that all nine certifications – and their corresponding training courses – have been included in the U.S. Department of Defense (DoD) Directive 8140 Cyber Workforce Qualification Provider Marketplace. The 8140 directive replaces the 8570 directive and went into effect on February 15, 2023. The DoD designed the new directive to “provide a targeted, role-based approach to identify, develop and qualify cyber personnel by leveraging the DoD Cyber Workforce Framework (DCWF).” What this means is that all personnel assigned to a position that is coded with a DCWF work role must meet the foundational, residential and continuous professional development qualification requirements outlined for each work role at the assigned proficiency level(s). Member Insights We asked a few of our member

In [15]:
df.to_csv("check_df.csv")

In [18]:
df.head()

,document_name,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count
0,log the right AWS Log Sources,1,Disclaimer: The views and opinions expressed i...,980,151,245.00
1,log the right AWS Log Sources,1,"In June 2021, Turkish beauty brand Cosmolog Ko...",1065,154,266.25
2,log the right AWS Log Sources,1,Critical AWS Logs and Why They Matter Integrat...,389,71,97.25
3,log the right AWS Log Sources,2,world scenarios and how they’ve helped me and ...,1185,178,296.25
4,log the right AWS Log Sources,2,They’re essential for tracking resource change...,806,118,201.50


In [16]:
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,86.00,86.00,86.00,86.00
mean,2.78,780.66,120.43,195.17
std,1.31,392.72,60.42,98.18
min,1.00,60.00,10.00,15.00
25%,2.00,418.25,67.25,104.56
50%,3.00,863.50,128.50,215.88
75%,4.00,1062.75,159.00,265.69
max,6.00,1734.00,261.00,433.50


## Embed & Retrieve

In [1]:
from FUNCTION_embed_n_retrieve import embed_n_retrieve

In [32]:
# Define helper function to print wrapped text 
import textwrap

def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

In [ ]:
MODEL_NAME_OR_PATH = "all-mpnet-base-v2"
DEVICE = "cpu"

In [ ]:
QUERY = "The launch of DeepSeek coincided with which AI-related milestone?"

top_results_dot_product = embed_n_retrieve(LST_pdf_docs=df["sentence_chunk"],
                                           QUERY=QUERY
                                           )

print(f"Query: '{QUERY}'\n")
print("Results:")
# Loop through zipped together scores and indicies from torch.topk
for score, idx in zip(top_results_dot_product[0], top_results_dot_product[1]):
    print(f"Score: {score:.4f}")
    # Print relevant sentence chunk (since the scores are in descending order, the most relevant chunk will be first)
    idx = int(idx)
    print(df.iloc[idx]["document_name"])
    print(f"page_num is {df.iloc[idx]["page_number"]}")
    print("Text:")
    print_wrapped(df.iloc[idx]["sentence_chunk"])
    print("\n")

Query: 'how much did the makers of DeepSeek claim the platform cost to train?'

Results:
Score: 0.6043
DeepSeek
page_num is 2
Text:
The big news with DeepSeek is, however, what the new R1 model cost – or at least
is claimed to have cost – to build and train. According to a paper from the
vendor, the figure was circa $5.6 million, which is a fraction of the $63
million claimed cost of training OpenAI’s GPT-4. A significant factor in the
relatively low cost is that DeepSeek, with its servers based in China, has no
access to the high-power processors that are preferred for AI training due to
U.S. export regulations. These rules, which have evolved over many years, were
reinforced in October 2022 and then again in 2023 with specific limitations
relating to the export of microprocessors that are particularly suited to AI
applications, the latter of these changes specifically quoting a U.S. government
report from February 2023: “China is rapidly expanding and improving its
artificial intelli

## {WIP} LLM to Generate Output to Directly Address Query